# Reversible Computing
---

In the [previous chapter](./01_01_digital_logic.ipynb), we introduced the concept of Boolean operators such as the binary operations **AND** and **OR**. A critical aspect of these elements is that they are **not** reversible. In other words, we cannot figure out the value of their inputs based solely on the value of the output. 

In case this is not entirely obvious, lets look again at the truth table for the **AND** operation:

| $b$ | $a$ | $b \land a$ |
| :-: | :-: | :-: |
| $0$ | $0$ | $0$ |
| $0$ | $1$ | $0$ |
| $1$ | $0$ | $0$ |
| $1$ | $1$ | $1$ |


Here we can see that if our output is $1$, we can confidently affirm that our inputs are $b = 1$ and $a = 1$. However, if the output is $0$, it is not possible to predict with certainty what the values of $a$ and $b$ are. This uncertainty is equivalent to a loss of information in our system.

[Reversible computing](https://en.wikipedia.org/wiki/Reversible_computing) was born from the idea that the loss of information in classical Boolean circuits is associated with power dissipation [[Landauer61]()]. In an effort to find more power-efficient ways to implement circuits, a model of reversible computation emerged [[Bennett73]()], and with it, a corresponding set of logical gate abstractions to perform these computations [[Toffoli80]()], [[Fredkin81]()].

It was very quickly realized that a model of computation at the quantum-mechanical level can be developed following these same rules [[Feynman85]()], and expanded to include non-classical operations such as superposition and entanglement [[Deutsch85]()], [[Margolus89]()]. Therefore, understanding the basic principles of reversible computing is an important step towards a model of quantum computation.

## 1. Reversible Logic

We begin by analyzing the basic operations we reviewed in the previous chapter in an effort to make them reversible. 
<br></br>

### 1.1 Reversible **NOT**
The first thing to note is that the **NOT** gate ***is*** reversible. This because we can confidently predict the input value directly from its output. If the output is $1$, we know for a fact the input was $0$, and vice versa. Furthermore, we can reverse the computation of a **NOT** gate by applying another **NOT** right after.

Now, because the symbol for the logic **NOT** gate is not symmetric, Feynman suggested changing it to simply using an X over a wire. However, from now on, we will instead use the symbol commonly utilized in quantum computing circuits, which is an X inside a box$^*$:


<img src="images\01_02_01_not_gates.png" align = "center" width="660"/>

For this same reason, we will now refer to the **NOT** gate as the **X** gate.

<span style="font-size: smaller;">$^*$**Aside**: The fact that Feynman used an X to denote the circuit diagram for a reversible **NOT** gate is merely coincidental with the notion of using an X as the symbol for its corresponding quantum gate. The reason why we call the quantum **NOT** an **X** gate has to do with its relationship with the Pauli-X matrix (a topic that we will cover in detail in a later chapter).</span>

### 1.2 Reversible **XOR**
The reversible version of the **XOR** gate is perhaps the most fundamental 2-bit operation in reversible computing, and one of the most widely used gates in quantum circuits. We can implement this operation by the used of a what is known as a controlled-X or **CX** gate. A **CX** gate has a control line and a target line. The value on the control line determines if an **X** gate on the target line gets applied or not: 
- If the control line is $0$, the target line is left unchanged.
- If the control is $1$, the **X** gate is activated, inverting the value of target line. 

The figure below shows the schematic representation of a **CX** gate where the top line is the control, and the bottom line is the target:

<img src="images\01_02_02_cx_gate.png" align = "center" width="210"/>

Following this logic, we can make a truth table for what the outputs $a'$ and $b'$ look like with respect to the inputs $a$ and $b$, which clearly shows that $b' = a \oplus b$:
| $a$ | $b$ | $a'$ | $b'$ |
| :-: | :-: | :-: | :-: |
| $0$ | $0$ | $0$ | $0$ |
| $0$ | $1$ | $0$ | $1$ |
| $1$ | $0$ | $1$ | $1$ |
| $1$ | $1$ | $1$ | $0$ |

The output $b'$ is the same as what would we get with a classical **XOR** gate, but the simple fact of carrying the input $a$ to the output $a'$ allows us to be able to always predict the value of both inputs ($a$ and $b$) directly from the outputs ($a'$ and $b'$). This should be clear from the fact that for each possible input there is only a single unique possible output.

Furthermore, the operation of a **CX** gate can be uncomputed (reversed) by applying another **CX** right after!. We can verify this with a few lines of code. Let's first verify our **CX** gate:

In [7]:
# define cx gate as a function:
def cx(a_in, b_in):
    a_out = a_in                 # a' is always equal to a
    
    # if control is 1 (a == 1): negate target (b' = b̅), else: leave b alone (b' = b).
    if a_in == 1:                
        b_out = (b_in + 1) % 2   # Adding 1 mod 2 flips a bit.
    else:
        b_out = b_in

    return (a_out, b_out)

print('CX gate:')
print('a | b | a\' | b\' |')

# Iterate over possible combinations of a and b
for a, b in [(0,0), (0,1), (1,0), (1,1)]:
    ap, bp = cx(a,b)
    print(f'{a} | {b} | {ap}  | {bp}  |')

CX gate:
a | b | a' | b' |
0 | 0 | 0  | 0  |
0 | 1 | 0  | 1  |
1 | 0 | 1  | 1  |
1 | 1 | 1  | 0  |


We can see that $a' = a$ and $b' = a \oplus b$, so our **CX** function seems to be working correctly. Let's now apply it twice:

In [10]:
print('Two CX gates:')
print('a | b | a\'\' | b\'\' |')

# Apply CX twice for all combination of inputs a, b.
for a, b in [(0,0), (0,1), (1,0), (1,1)]:
    ai, bi = cx(a,b)
    ap, bp = cx(ai,bi)
    print(f'{a} | {b} | {ap}   | {bp}   |')

Two CX gates:
a | b | a'' | b'' |
0 | 0 | 0   | 0   |
0 | 1 | 0   | 1   |
1 | 0 | 1   | 0   |
1 | 1 | 1   | 1   |


We can see that after applying to **CX** gates in a row, the inputs $a$ and $b$ now match the ouputs $a''$ and $b''$, which means the computation has been reversed.

### 1.2 Reversible **COPY**
A reversible implementation of the **COPY** or **FANOUT** operation can be easily implemented by simply taking a **CX** and always setting the input of the target line to $0$:

<img src="images\01_02_03_copy_gate.png" align = "center" width="210"/>

We can see from the **CX** truth table that if $b = 0$, then $b' = a' = a$.

### 1.3 Reversible **AND**

Recall that for an **AND** gate, three of the four possible outputs are equal to $0$:
| $a$ | $b$ | $a \land b$ |
| :-: | :-: | :-: |
| $0$ | $0$ | $0$ |
| $0$ | $1$ | $0$ |
| $1$ | $0$ | $0$ |
| $1$ | $1$ | $1$ |

This poses a challenge because it is then not possible to predict **both** of the input values using the output ($a \land b$), and only one of the inputs ($a$ or $b$). The consequence of this is that we need a 3-bit gate in order to construct a reversible **AND** operation. The way to do this is by the use of a controlled-controlled-X or **CCX** gate, also often referred as a Toffoli gate. 

Example of footnote.<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)

Example of footnote.<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2)


<a name="cite_note-1"></a>1. [^](#cite_ref-1) footnote 1

<a name="cite_note-2"></a>2. [^](#cite_ref-2) footnote 2